In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# tried office365 API to access via url and didn't work due to issues with the installation of package
# download the file and save it in the same path
#url = 'https://lanecouncilofgovernments-my.sharepoint.com/:x:/g/personal/sshoaf_lcog_org/EaiRosQicVhNrRvpKjgrZ9cByrC1Z6D7esomrXNvMsHxUw?e=VFaLaP'
xl = pd.ExcelFile('Working DRAFT 2045 Project List.xlsx')
sheetNames45 = xl.sheet_names

In [3]:
def getSheetnames(excel='Working DRAFT 2045 Project List.xlsx',
                 pattern='Auto Constrained'):
    xl = pd.ExcelFile(excel)
    sheetList = xl.sheet_names
    return [sheet for sheet in sheetList if re.match(pattern, sheet)]

In [4]:
def getRTPid(excel='Working DRAFT 2045 Project List.xlsx',
             sheet_name='Auto Constrained - Arterial Lin'):
    xl = pd.ExcelFile(excel)
    df = xl.parse(sheet_name)
    name = df['Name'][0].split(":")[1].lstrip()
    df = df.drop(labels=0, axis=0)
    rtplist = df['RTP #'].unique()
    strlist = [item for item in rtplist if type(item) is str]
    if len(strlist) > 0:
        for s in strlist:
            rtplist = rtplist[rtplist != s]
    a = rtplist.astype(float)
    b = a[~np.isnan(a)].astype(int).tolist()
    res = [int(ele) for ele in b if ele > 0]
    return name, res

In [5]:
# check the project list in 2040
xl = pd.ExcelFile('2040 Project List_Consolidated draft with AQ (ORIGINAL).xlsx')
sheetNames40=xl.sheet_names

In [6]:
[name for name in sheetNames45 if name not in sheetNames40]

['Auto Constrained - Added Fwy', 'Auto Illustrative - Urban Stnds']

In [7]:
df = xl.parse('Auto Constrained - Arterial Lin', skiprows=3)
df.head()

,Name,Geographic Limits,Description,Primary\nJurisdiction,Air Quality Status,Estimated Cost (2011),Estimated Cost (2016),Estimated Year of Construction\n(4-Year Window),Year of Construction\nCost Range,Unnamed: 9,Length,RTP #,Federal Functional Class,Jurisdictional Project #,Comments,Unnamed: 15,Unnamed: 16,Unnamed: 17,Inflation Rate
0,Project Category: New Arterial Link or Interc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.1
1,Eugene-Springfield Highway (SR-126),at Main Street,Construct interchange,ODOT,non-exempt; regionally significant,50000000.0,50000000.0,2030-2034,7.666397e+07,8.662156e+07,0.00,27.0,Other Freeways and Expressways,NaN,NaN,NaN,NaN,NaN,NaN
2,Eugene-Springfield Highway (SR-126),at 52nd Street,Construct interchange,ODOT,non-exempt; regionally significant,40000000.0,40000000.0,2025-2029,5.264874e+07,5.948708e+07,0.00,30.0,Other Freeways and Expressways,NaN,NaN,NaN,NaN,NaN,NaN
3,Beltline Local Arterial Bridge,Beaver Street to Delta Highway,Construct new arterial bridge with up to 4 lan...,"ODOT, Lane County, City of Eugene",non-exempt; regionally significant; project of...,32000000.0,83000000.0,2025-2029,1.092461e+08,1.234357e+08,0.95,512.0,minor arterial,ETSP MM-3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
def addCategory(df):
    name = df['Name'][0].split(":")[1].lstrip()
    df = df.drop(labels=0, axis=0)
    df = df[df.Name.astype(str) != 'nan']
    #df = df[df.columns.drop(list(df.filter(regex='Unnamed')))]
    s = pd.Series([name])
    df['Category'] = list(s.repeat(df.shape[0]))
    return df

In [19]:
def readTable(sheetName='Auto Constrained - Arterial Lin',
              year=2040):
    if year == 2040:
        table='2040 Project List_Consolidated draft with AQ (ORIGINAL).xlsx'
    else:
        table='Working DRAFT 2045 Project List.xlsx'
        
    xl = pd.ExcelFile(table)
    
    if sheetName == 'Transit Constrained':
        if year == 2040:
            df1 = xl.parse(sheetName, skiprows=3, nrows=6)
            df1 = addCategory(df1)
            columns = df1.columns

            df2 = xl.parse(sheetName, skiprows=10, nrows=9)
            df2.columns = list(columns[0:(len(columns)-1)])
            df2 = addCategory(df2)

            df3 = xl.parse(sheetName, skiprows=20, nrows=6)
            df3.columns = list(columns[0:(len(columns)-1)])
            df3 = addCategory(df3)       
        else:
            df1 = xl.parse(sheetName, nrows=5)
            df1 = addCategory(df1)
            columns = df1.columns

            df2 = xl.parse(sheetName, skiprows=7, nrows=9)
            df2.columns = list(columns[0:(len(columns)-1)])
            df2 = addCategory(df2)

            df3 = xl.parse(sheetName, skiprows=17, nrows=6)
            df3.columns = list(columns[0:(len(columns)-1)])
            df3 = addCategory(df3)       

        df = pd.concat([df1, df2, df3], ignore_index=True)
        df.rename(columns={"Unnamed: 8": "Year of Construction Cost Max"}, inplace=True)
        
    else:
        df = xl.parse(sheetName)
        if len([col for col in df.columns if 'Unnamed' in col]) > 5:
            df = xl.parse(sheetName,  skiprows=3)
        if year == 2045:
            if sheetName in ['Bike Illustrative - withRd', 'Bike Illustrative - onstreet w', 
                             'Bike Illustrative - onstreet w', 'Bike Illustrative onstreet wout']:
                df.rename(columns={"Unnamed: 7": "Year of Construction Cost Max"}, inplace=True)
            else:
                df.rename(columns={"Unnamed: 8": "Year of Construction Cost Max"}, inplace=True)
        else:
            if sheetName in ['Auto Constrained - Study', 'Bike Constrained - wRd', 
                             'Bike Constrained - onstreet w', 'Bike Constrained - onstreet wou',
                             'Bike Illustrative - woutRD', 'Bike Illustrative - onstreet w',
                             'Bike Illustrative onstreet wout']:
                df.rename(columns={"Unnamed: 8": "Year of Construction Cost Max"}, inplace=True)
            elif sheetName == 'Transit Illustrative':
                df.rename(columns={"Unnamed: 7": "Year of Construction Cost Max"}, inplace=True)
            else:
                df.rename(columns={"Unnamed: 9": "Year of Construction Cost Max"}, inplace=True)
        if df.shape[0] == 0:
            pass
        else:
            df = addCategory(df)
    if year == 2040:    
        df.rename(columns={"Year of Construction\nCost Range": "Year of Construction Cost Min"}, inplace=True)
    else:
        df.rename(columns={"Year of Construction Range": "Year of Construction Cost Min",
                           "Year of Construction\nCost Range": "Year of Construction Cost Min",
                           "Year of Construction \nCost Range": "Year of Construction Cost Min"}, inplace=True)
            
    df = df[df.columns.drop(list(df.filter(regex='Unnamed')))]
    df.columns = df.columns.str.replace(' ', '')
    df.columns = df.columns.str.replace('\n', '')
    if 'EstimatedYearofStudy(4-YearWindow)' in list(df.columns):
        df.rename(columns={"EstimatedYearofStudy(4-YearWindow)": "EstimatedYearofConstruction(4-YearWindow)"}, 
                  inplace=True) 
    return df  

In [10]:
table='Working DRAFT 2045 Project List.xlsx'

In [11]:
xl = pd.ExcelFile(table)

In [12]:
df = xl.parse('Transit Constrained', nrows=5)
df

,Name,Geogrpahic Limits,Description,Primary Jurisdiction,Air Quality Status,Estimated Cost (2021),Estimated Year of Construction (4-Year Window),Year of Construction Range,Unnamed: 8,Length,RTP #
0,Project Category: Buses and Bus Maintenance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bus Purchases,NaN,New & Replacement Buses,Lane Transit District,Non-Exempt,67790000.0,2021-2025,6.989149e+07,7.896942e+07,-,1110.0
2,Bus Purchases,NaN,New & Replacement Buses,Lane Transit District,Non-Exempt,31460000.0,2026-2030,3.778424e+07,4.269189e+07,-,1110.0
3,Bus Purchases,NaN,New & Replacement Buses,Lane Transit District,Non-Exempt,55000000.0,2031-2035,7.694989e+07,8.694461e+07,-,1110.0
4,Bus Purchases,NaN,New & Replacement Buses,Lane Transit District,Non-Exempt,55000000.0,2036-2040,8.963989e+07,1.012829e+08,-,1110.0


In [22]:
def combineTables(year=2040):
    if year == 2040:
        table='2040 Project List_Consolidated draft with AQ (ORIGINAL).xlsx'
    else:
        table='Working DRAFT 2045 Project List.xlsx'
        
    xl = pd.ExcelFile(table)
    sheetNames = xl.sheet_names
    sheetNames = [sheetnm for sheetnm in sheetNames if sheetnm != 'Table Data']
    for sheetName in sheetNames:
        print(sheetName)
        if sheetName == sheetNames[0]:
            df = readTable(sheetName=sheetName, year=year)
        else:
            ndf = readTable(sheetName=sheetName, year=year)
            if ndf.shape[0] == 0:
                pass
            else:
                selectedColumns = [a for a in list(ndf.columns) if a in list(df.columns)]
                ndf = ndf[selectedColumns]
                df = df[selectedColumns]
                df = df.append(ndf, ignore_index=True)
                #print(list(df.columns))
                #print('EstimatedCost(2016)' in list(df.columns))
    return df

for var in ['AirQualityStatus', 'EstimatedCost(2011)', 
            'EstimatedCost(2016)', 'EstimatedYearofConstruction(4-YearWindow)', 
            'JurisdictionalProject#']:
    print(var)
    print(var in list(df.columns))

year=2040
if year == 2040:
    table='2040 Project List_Consolidated draft with AQ (ORIGINAL).xlsx'
else:
    table='Working DRAFT 2045 Project List.xlsx'

xl = pd.ExcelFile(table)
sheetNames = xl.sheet_names
[sheetnm for sheetnm in sheetNames if sheetnm != 'Table Data']

table='Working DRAFT 2045 Project List.xlsx'
xl = pd.ExcelFile(table)
sheetNames = xl.sheet_names
[sheetnm for sheetnm in sheetNames if sheetnm != 'Table Data']

table='Working DRAFT 2045 Project List.xlsx'
xl = pd.ExcelFile(table)
sheetNames = xl.sheet_names
for sheetName in [sheetnm for sheetnm in sheetNames if sheetnm != 'Table Data']:
    df = xl.parse(sheetName)
    #df = readTable(sheetName='Auto Constrained - Arterial Lin', year=2045)
    print(sheetName)
    print(df.columns)  

In [23]:
df = combineTables() 

Auto Constrained - Arterial Lin
Auto Constrained - Added Freewa
Auto Constrained - Arterial Cap
Auto Constrained - New Collecto
Auto Constrained - Urban Stnds
Auto Constrained - Study
Auto Constrained - TOD
Auto Illustrative - New Arteria
Auto Illustrative - New Collect
Auto Illustrative - Art Cap
Auto Illustrative - Added Freew
Auto Illustrative - Urban Stds
Transit Constrained
Transit Illustrative
Bike Constrained - woutRd
Bike Constrained - wRd
Bike Constrained - onstreet w
Bike Constrained - onstreet wou
Bike Illustrative - woutRD
Bike Illustrative - withRd
Bike Illustrative - onstreet w
Bike Illustrative onstreet wout


In [24]:
df.head()

,Name,GeographicLimits,PrimaryJurisdiction,EstimatedCost(2016),EstimatedYearofConstruction(4-YearWindow),YearofConstructionCostMin,YearofConstructionCostMax,RTP#,Category
0,Eugene-Springfield Highway (SR-126),at Main Street,ODOT,50000000.0,2030-2034,7.666397e+07,8.662156e+07,27.0,New Arterial Link or Interchange
1,Eugene-Springfield Highway (SR-126),at 52nd Street,ODOT,40000000.0,2025-2029,5.264874e+07,5.948708e+07,30.0,New Arterial Link or Interchange
2,Beltline Local Arterial Bridge,Beaver Street to Delta Highway,"ODOT, Lane County, City of Eugene",83000000.0,2025-2029,1.092461e+08,1.234357e+08,512.0,New Arterial Link or Interchange
3,Delta/ Beltline Interchange,NaN,ODOT,20000000.0,2020-2024,2.259772e+07,2.553285e+07,638.0,Added Freeway Lanes or Major Interchange Impro...
4,Randy Pape Beltline Highway,Roosevelt Boulevard to W. 11th Avenue,"ODOT, Eugene",28100000.0,2030-2034,4.308515e+07,4.868131e+07,312.0,Added Freeway Lanes or Major Interchange Impro...


In [25]:
df = combineTables(year=2045)

Auto Constrained - Arterial Lin
Auto Constrained - Added Fwy
Auto Constrained - Arterial Cap
Auto Constrained - New Collecto
Auto Constrained - Urban Stnds
Auto Constrained - Study
Auto Constrained - TOD
Auto Illustrative - New Arteria
Auto Illustrative - New Collect
Auto Illustrative - Art Cap
Auto Illustrative - Added Freew
Auto Illustrative - Urban Stnds
Transit Constrained
Transit Illustrative
Bike Constrained - woutRd
Bike Constrained - wRd
Bike Constrained - onstreet w
Bike Constrained - onstreet wou
Bike Illustrative - woutRD
Bike Illustrative - withRd
Bike Illustrative - onstreet w
Bike Illustrative onstreet wout


In [26]:
df.head()

,Name,Description,PrimaryJurisdiction,EstimatedYearofConstruction(4-YearWindow),YearofConstructionCostMin,YearofConstructionCostMax,Length,RTP#,Category
0,Beltline Local Arterial Bridge,Construct new 2-lane arterial bridge over the ...,"ODOT, Lane County, City of Eugene",2025-2029,NaN,NaN,0.95,512,New Arterial Link or Interchange
1,Eugene-Springfield Highway (SR-126),Construct interchange,ODOT,2030-2034,NaN,NaN,0.0,27,New Arterial Link or Interchange
2,Eugene-Springfield Highway (SR-126),Construct interchange,ODOT,2025-2029,NaN,NaN,0.0,30,New Arterial Link or Interchange
3,Randy Pape Beltline Highway,Add lanes on Belltine Highway and provide inte...,"ODOT, Eugene",2030-2034,NaN,NaN,1.1,312.0,Added Freeway Lanes or Major Interchange Impro...
4,Delta/Beltline Interchange,Interim/safety improvements; replace/revise ex...,ODOT,2020-2024,NaN,NaN,0.25,638.0,Added Freeway Lanes or Major Interchange Impro...
